## Overview

This notebook loads the movie rating data from DSX's local storage then it trains an *alternating least square* (ALS) model using Spark's Machine Learning library (MLlib).<br>
For more information on Spark ALS, see here:
- http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html#collaborative-filtering
- https://github.com/jadianes/spark-movie-lens

## Load the data

Again, let's preview the data

In [7]:
!head -3 ratings.dat
!echo
!tail -3 ratings.dat

1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968

6040::562::5::956704746
6040::1096::4::956715648
6040::1097::4::956715569


The data is in the format:
    
`UserID::MovieID::Rating::Timestamp`
                        
Now load it into an RDD

In [8]:
from pyspark.mllib.recommendation import Rating

ratingsRDD = sc.textFile('ratings.dat') \
               .map(lambda l: l.split("::")) \
               .map(lambda p: Rating(
                                  user = int(p[0]), 
                                  product = int(p[1]),
                                  rating = float(p[2]), 
                                  )).cache()

It's useful to check some highlevel statistics on the data. At this point, I'm mainly interested in the row count.

In [9]:
ratingsRDD.toDF().describe().show()

+-------+------------------+------------------+------------------+
|summary|              user|           product|            rating|
+-------+------------------+------------------+------------------+
|  count|           1000209|           1000209|           1000209|
|   mean| 3024.512347919285|1865.5398981612843| 3.581564453029317|
| stddev|1728.4126948999951|1096.0406894572552|1.1171018453732544|
|    min|                 1|                 1|               1.0|
|    max|              6040|              3952|               5.0|
+-------+------------------+------------------+------------------+



## Split into training and testing

Next we split the data into training and testing data sets

In [10]:
(training, test) = ratingsRDD.randomSplit([0.8, 0.2])

numTraining = training.count()
numTest = test.count()

# verify row counts for each dataset
print("Total: {0}, Training: {1}, test: {2}".format(ratingsRDD.count(), numTraining, numTest))

Total: 1000209, Training: 800790, test: 199419


## Build the recommendation model using ALS on the training data

I've chosen some values for the ALS parameters.  You should probaly experiment with different values.

In [11]:
from pyspark.mllib.recommendation import ALS

rank = 50
numIterations = 20
lambdaParam = 0.1
model = ALS.train(training, rank, numIterations, lambdaParam)

Let's save the model ...

In [207]:
pf = model.productFeatures()
Vt = np.matrix(np.asarray(pf.values().collect()))

def save_Vt(Vt, outfile):
    Vt.dump(outfile.name)
    # TODO upload outfile to Cloudant database
    
def load_Vt(outfile):
    # TODO download outfile from Cloudant database
    Vt = np.load(outfile.name)
    return Vt

def simulate_saving_Vt_to_remote_storage(Vt):
    from tempfile import NamedTemporaryFile
    outfile = NamedTemporaryFile()
    save_Vt(Vt, outfile)
    outfile.seek(0) # Only needed here to simulate closing & reopening file
    return load_Vt(outfile)


In [208]:
Vt = simulate_saving_Vt_to_remote_storage(Vt)

In [209]:
full_u = np.zeros(3676)
full_u[1] = 5 # user has rated product_id:1 = 5
recommendations = full_u*Vt*Vt.T

print("predicted rating value", np.sort(recommendations)[:,-10:])

top_ten_recommended_product_ids = np.where(recommendations >= np.sort(recommendations)[:,-10:].min())[1]

print("predict rating prod_id", np.array_repr(top_ten_recommended_product_ids))

('predicted rating value', matrix([[ 14.54882126,  14.56155905,  14.56796413,  14.60397112,
          14.61495758,  14.88292751,  15.15791346,  15.33834965,
          16.05929494,  16.5434285 ]]))
('predict rating prod_id', 'array([ 280,  714, 1856, 2211, 2302, 2661, 2832, 3028, 3094, 3330])')
